In [1]:
import pandas as pd
import re
import json
import math
from itertools import permutations
from IPython.display import clear_output
from time import sleep
from abc import ABC, abstractmethod, abstractproperty

from grid_builder import *
from grid import *
from tile import *
from tiles import *
from destructable import *
from unit import *
from command import *
from ability import *
from shell import *
from beam import *
from game import *

In [2]:
gb = GridBuilder()

gb.place( WaterTile(),      (0,4) )
gb.place( WaterTile(),      (0,5) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (0,6) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (0,7) )

gb.place( Destructable(CorporateTile(), GroundTile(), 1), (1,1) )
gb.place( ForestTile(),     (1,2) )
gb.place( Destructable(CivilianTile(), GroundTile(), 1),  (1,3) )
gb.place( Destructable(CivilianTile(), GroundTile(), 1),  (1,4) )
gb.place( WaterTile(),      (1,5) )
gb.place( WaterTile(),      (1,6) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (1,7) )

gb.place( ForestTile(),     (2,3) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (2,5) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (2,6) )
gb.place( WaterTile(),      (2,7) )

gb.place( Destructable(CivilianTile(), GroundTile(), 1),  (3,1) )
gb.place( WaterTile(),      (3,7) )

gb.place( Destructable(CivilianTile(), GroundTile(), 2),  (4,1) )
gb.place( Destructable(CivilianTile(), GroundTile(), 2),  (4,5) )
gb.place( Destructable(CivilianTile(), GroundTile(), 2),  (4,6) )

gb.place( ForestFireTile(), (5,1) )
gb.place( ForestFireTile(), (5,2) )
gb.place( SpawnTile(),      (5,4) )
gb.place( ForestTile(),     (5,7) )

gb.place( Destructable(MountainTile(), GroundTile(), 3),   (6,0) )
gb.place( SpawnTile(),      (6,2) )
gb.place( ForestFireTile(), (6,3) )
gb.place( ForestFireTile(), (6,4) )

gb.place( Destructable(MountainTile(), GroundTile(), 3),   (7,0) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (7,1) )
gb.place( ForestTile(),     (7,3) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (7,7) )

gb.place_on_tile( Mech("Siege Mech"),     (3,2) )
gb.place_on_tile( Mech("Artillery Mech"), (5,3) )
gb.place_on_tile( Mech("Boulder Mech"),   (7,3) )
gb.place_on_tile( Vek("Psy"),             (4,2) )
gb.place_on_tile( Vek("Alpha Firefly"),   (4,4) )
gb.place_on_tile( Vek("Scarab"),          (6,1) )
gb.place_on_tile( Vek("Firefly"),         (6,5) )
gb.place_on_tile( Unit("Boulder", max_health=1, health=1, moves=0), (7,5) )

grid = gb.to_grid()

for mech in grid.mechs:
    mech.add(Move(mech,grid))
    mech.add(Repair(mech))
    
for vek in grid.veks:
    vek.add(Move(vek, grid))

    
siege_mech     = grid.units['Siege Mech']
artillery_mech = grid.units['Artillery Mech']
boulder_mech   = grid.units['Boulder Mech']

firefly        = grid.units['Firefly']
alpha_firefly  = grid.units['Alpha Firefly']
scarab         = grid.units['Scarab']
psy            = grid.units['Psy']


siege_mech.add(Artillery(siege_mech, grid, ClusterShell, 2))
boulder_mech.add(Artillery(boulder_mech, grid, BoulderShell, 2))
artillery_mech.add(Artillery(artillery_mech, grid, RegularShell, 1))
    
grid.show()


,0,1,2,3,4,5,6,7
0,🟢,🟢,🟢,🟢,💦,💦,🗻,🗻
1,🟢,🏢 ϟ,🌲,🏘️ ϟ,🏘️ ϟ,💦,💦,🗻
2,🟢,🟢,🟢,🌲,🟢,🗻,🗻,💦
3,🟢,🏘️ ϟ,Siege Mech ♡♡♡♡,🟢,🟢,🟢,🟢,💦
4,🟢,🏘️ ϟϟ,Psy ♡♡♡♡,🟢,Alpha Firefly ♡♡♡♡,🏘️ ϟϟ,🏘️ ϟϟ,🟢
5,🟢,🔥,🔥,Artillery Mech ♡♡♡♡,⬆️,🟢,🟢,🌲
6,🗻,Scarab ♡♡♡♡,⬆️,🔥,🔥,Firefly ♡♡♡♡,🟢,🟢
7,🗻,🗻,🟢,Boulder Mech ♡♡♡♡,🟢,Boulder ♡,🟢,🗻


In [3]:
Game(grid)

Round 1


,0,1,2,3,4,5,6,7
0,🟢,🟢,🟢,🟢,💦,💦,🗻,🗻
1,🟢,🏢 ϟ,🌲,🏘️ ϟ,🏘️ ϟ,💦,💦,🗻
2,🟢,🟢,🟢,🌲,🟢,🗻,🗻,💦
3,🟢,🏘️ ϟ,Siege Mech ♡♡♡♡,🟢,🟢,🟢,🟢,💦
4,🟢,🏘️ ϟϟ,Psy ♡♡♡♡,🟢,Alpha Firefly ♡♡♡♡,🏘️ ϟϟ,🏘️ ϟϟ,🟢
5,🟢,🔥,🔥,Artillery Mech ♡♡♡♡,⬆️,🟢,🟢,🌲
6,🗻,Scarab ♡♡♡♡,⬆️,🔥,🔥,Firefly ♡♡♡♡,🟢,🟢
7,🗻,🗻,🟢,Boulder Mech ♡♡♡♡,🟢,Boulder ♡,🟢,🗻


1) Siege Mech ♡♡♡♡
2) Artillery Mech ♡♡♡♡
3) Boulder Mech ♡♡♡♡
Which Mech to Move? 1

Siege Mech ♡♡♡♡ moves:
1) MOVE (3, 2) to (1, 2)
2) MOVE (3, 2) to (3, 2)
3) MOVE (3, 2) to (3, 3)
4) MOVE (3, 2) to (0, 2)
5) MOVE (3, 2) to (2, 1)
6) MOVE (3, 2) to (2, 0)
7) MOVE (3, 2) to (2, 3)
8) MOVE (3, 2) to (4, 3)
9) MOVE (3, 2) to (2, 2)
10) MOVE (3, 2) to (3, 4)
11) MOVE (3, 2) to (2, 4)
12) MOVE (3, 2) to (3, 5)
13) HEAL +1
14) CLUSTER SHELL at (0, 2)
15) CLUSTER SHELL at (1, 2)
16) CLUSTER SHELL at (5, 2)
17) CLUSTER SHELL at (6, 2)
18) CLUSTER SHELL at (7, 2)
19) CLUSTER SHELL at (3, 0)
20) CLUSTER SHELL at (3, 4)
21) CLUSTER SHELL at (3, 5)
22) CLUSTER SHELL at (3, 6)
23) CLUSTER SHELL at (3, 7)
Which Move? 23


,0,1,2,3,4,5,6,7
0,🟢,🟢,🟢,🟢,💦,💦,🗻,🗻
1,🟢,🏢 ϟ,🌲,🏘️ ϟ,🏘️ ϟ,💦,💦,🗻
2,🟢,🟢,🟢,🌲,🟢,🗻,🗻,💦
3,🟢,🏘️ ϟ,Siege Mech ♡♡♡♡,🟢,🟢,🟢,🟢,💦
4,🟢,🏘️ ϟϟ,Psy ♡♡♡♡,🟢,Alpha Firefly ♡♡♡♡,🏘️ ϟϟ,🏘️ ϟϟ,🟢
5,🟢,🔥,🔥,Artillery Mech ♡♡♡♡,⬆️,🟢,🟢,🌲
6,🗻,Scarab ♡♡♡♡,⬆️,🔥,🔥,Firefly ♡♡♡♡,🟢,🟢
7,🗻,🗻,🟢,Boulder Mech ♡♡♡♡,🟢,Boulder ♡,🟢,🗻


,0,1,2,3,4,5,6,7
0,🟢,🟢,🟢,🟢,💦,💦,🗻,🗻
1,🟢,🏢 ϟ,🌲,🏘️ ϟ,🏘️ ϟ,💦,💦,🗻
2,🟢,🟢,🟢,🌲,🟢,🗻,🗻,💦
3,🟢,🏘️ ϟ,Siege Mech ♡♡♡♡,🟢,🟢,🟢,🟢,💦
4,🟢,🏘️ ϟϟ,Psy ♡♡♡♡,🟢,Alpha Firefly ♡♡♡♡,🏘️ ϟϟ,🏘️ ϟϟ,🟢
5,🟢,🔥,🔥,Artillery Mech ♡♡♡♡,⬆️,🟢,🟢,🌲
6,🗻,Scarab ♡♡♡♡,⬆️,🔥,🔥,Firefly ♡♡♡♡,🟢,🟢
7,🗻,🗻,🟢,Boulder Mech ♡♡♡♡,🟢,Boulder ♡,🟢,🗻


...

,0,1,2,3,4,5,6,7
0,🟢,🟢,🟢,🟢,💦,💦,🗻,🗻
1,🟢,🏢 ϟ,🌲,🏘️ ϟ,🏘️ ϟ,💦,💦,🗻
2,🟢,🟢,🟢,🌲,🟢,🗻,🗻,💦
3,🟢,🏘️ ϟ,Siege Mech ♡♡♡♡,🟢,🟢,🟢,🟢,💦
4,🟢,🏘️ ϟϟ,Psy ♡♡♡♡,🟢,Alpha Firefly ♡♡♡♡,🏘️ ϟϟ,🏘️ ϟϟ,🟢
5,🟢,🔥,🔥,Artillery Mech ♡♡♡♡,⬆️,🟢,🟢,🌲
6,🗻,Scarab ♡♡♡♡,⬆️,🔥,🔥,Firefly ♡♡♡♡,🟢,🟢
7,🗻,🗻,🟢,Boulder Mech ♡♡♡♡,🟢,Boulder ♡,🟢,🗻


next round (y/n)?y
Round 2


,0,1,2,3,4,5,6,7
0,🟢,🟢,🟢,🟢,💦,💦,🗻,🗻
1,🟢,🏢 ϟ,🌲,🏘️ ϟ,🏘️ ϟ,💦,💦,🗻
2,🟢,🟢,🟢,🌲,🟢,🗻,🗻,💦
3,🟢,🏘️ ϟ,Siege Mech ♡♡♡♡,🟢,🟢,🟢,🟢,💦
4,🟢,🏘️ ϟϟ,Psy ♡♡♡♡,🟢,Alpha Firefly ♡♡♡♡,🏘️ ϟϟ,🏘️ ϟϟ,🟢
5,🟢,🔥,🔥,Artillery Mech ♡♡♡♡,⬆️,🟢,🟢,🌲
6,🗻,Scarab ♡♡♡♡,⬆️,🔥,🔥,Firefly ♡♡♡♡,🟢,🟢
7,🗻,🗻,🟢,Boulder Mech ♡♡♡♡,🟢,Boulder ♡,🟢,🗻


1) Siege Mech ♡♡♡♡
2) Artillery Mech ♡♡♡♡
3) Boulder Mech ♡♡♡♡
Which Mech to Move? 

None moves:


AttributeError: 'NoneType' object has no attribute 'gen_actions'